# Assignment_11

## 代码复现

### Dynamic Programming

### 基本思想：
>把一个较复杂的问题按照阶段划分，分解为若干个较小的局部问题，然后按照局部问题的递推关系，依次作出一系列决策，直至整个问题达到总体最优的目标。

### 动态规划包含三个重要的概念：

>* 最优子结构
>* 边界
>* 状态转移方程

### 解题的一般步骤是：

>1. 找出最优解的性质，刻画其结构特征和最优子结构特征；
>2. 递归地定义最优值，刻画原问题解与子问题解间的关系；
>3. 以自底向上的方式计算出各个子问题、原问题的最优值，并避免子问题的重复计算；
>4. 根据计算最优值时得到的信息，构造最优解。
### 使用动态规划特征：

>1. 求一个问题的最优解
>2. 大问题可以分解为子问题，子问题还有重叠的更小的子问题
>3. 整体问题最优解取决于子问题的最优解（状态转移方程）
>4. 从上往下分析问题，从下往上解决问题
>5. 讨论底层的边界问题


#### 问题一：现在要求一段长度N的钢筋最佳切法，使得利益最大化

In [1]:
from collections import defaultdict

In [2]:
# 钢筋米数对应价目表
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 35]
price = defaultdict(int)

In [3]:
# 这里用defaultdict用处是当输入不存在的键返回0
for i,p in enumerate(original_price):
    #print(i+1,p)
    price[i+1] = p

In [4]:
# 显示钢筋米数对应的价格
price[11]

35

In [5]:
# 钢筋米数对应价目表
price

defaultdict(int,
            {1: 1,
             2: 5,
             3: 8,
             4: 9,
             5: 10,
             6: 17,
             7: 17,
             8: 20,
             9: 24,
             10: 30,
             11: 35})

In [6]:
# 求出最大价钱，但是切法却没有求出
def r(n):  
    return max([price[n]] + [r(i) + r(n-i) for i in range(1,n)])

In [7]:
%%time

r(8)

Wall time: 997 µs


22

In [8]:
%%time
r(15)

Wall time: 2.94 s


45

In [9]:
import time

In [10]:
#@get_time
# 斐波纳契数列
def fibonacci(n):
    if n  <= 2:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

In [11]:
start = time.time()
print(fibonacci(34))
end = time.time()
print(end-start)

5702887
0.927476167678833


In [12]:
mem = defaultdict()
def fibonacci_op(n):
    if n in mem:
        return mem[n]
    else: 
        if n <= 2:
            mem[n] = 1
            return n
        else:
            result = fibonacci_op(n-1) + fibonacci_op(n-2)
            mem[n] = result
            return result

In [13]:
start = time.time()
print(fibonacci_op(64))
end = time.time()
print(end-start)

14662949395604
0.0


# Analysis: How to optimize

## A Simpler Problem

### Decorator

In [14]:
def get_time(func):
    def wrapper(*args):
        start = time.time()
        func(*args)
        end = time.time()
        print('used time : {}'.format(end-start))
    return wrapper

In [15]:
from functools import wraps

In [16]:
# 减少重复计算次数，把已经计算过的存入字典，再次访问时先查字典，
# 如果有就直接读取，没有再去计算
def memo(f):
    memo.already_computed = {}
    @wraps(f)
    def _wrap(arg):
        if arg in memo.already_computed:
            result = memo.already_computed[arg]
        else:
            result = f(arg)
            memo.already_computed[arg] = result
        return result
    return _wrap

# We use this method to solve Cut Rod probelm¶

In [17]:
# solution记录每个长度的最佳切法
solution = {}

In [18]:
@memo
def r(n):
    """
    Args: n is the iron length
    Return: the max revenue 
    """
    max_price, max_split = max([(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], key=lambda x: x[0])

    solution[n] = (n - max_split, max_split)
    
    return max_price

In [19]:
r(20)

60

In [20]:
price

defaultdict(int,
            {1: 1,
             2: 5,
             3: 8,
             4: 9,
             5: 10,
             6: 17,
             7: 17,
             8: 20,
             9: 24,
             10: 30,
             11: 35,
             15: 0,
             14: 0,
             13: 0,
             12: 0,
             20: 0,
             19: 0,
             18: 0,
             17: 0,
             16: 0})

In [21]:
solution

{1: (1, 0),
 2: (2, 0),
 3: (3, 0),
 4: (2, 2),
 5: (3, 2),
 6: (6, 0),
 7: (6, 1),
 8: (6, 2),
 9: (6, 3),
 10: (10, 0),
 11: (11, 0),
 12: (11, 1),
 13: (11, 2),
 14: (11, 3),
 15: (13, 2),
 16: (14, 2),
 17: (11, 6),
 18: (17, 1),
 19: (17, 2),
 20: (17, 3)}

# How do we parse solution?¶

In [22]:
def parse_solution(n):
    left_split, right_split = solution[n]
    
    if right_split == 0: return [left_split]
    
    return parse_solution(left_split) + parse_solution(right_split)

In [23]:
r(20)

60

In [24]:
parse_solution(20)

[11, 6, 3]

# Edit Distance

In [25]:
solution = {}

In [26]:
from functools import lru_cache

In [27]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    if len(string1)==0: return len(string2)
    if len(string2)==0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2))
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB: {} = {}'\
                        .format(tail_s1, tail_s2))
        
    candidates.append(both_forward)    
    min_distance, operation = min(candidates, key = lambda x: x[0])   
    solution[(string1, string2)] = operation
    return min_distance

In [28]:
edit_distance('ABCDECG','ABCEFkk')

4

In [29]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCE'): 'ADD E',
 ('A', 'ABCEF'): 'ADD F',
 ('A', 'ABCEFk'): 'ADD k',
 ('A', 'ABCEFkk'): 'ADD k',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCE'): 'ADD E',
 ('AB', 'ABCEF'): 'ADD F',
 ('AB', 'ABCEFk'): 'ADD k',
 ('AB', 'ABCEFkk'): 'ADD k',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCE'): 'ADD E',
 ('ABC', 'ABCEF'): 'ADD F',
 ('ABC', 'ABCEFk'): 'ADD k',
 ('ABC', 'ABCEFkk'): 'ADD k',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCE'): 'SUB: D = E',
 ('ABCD', 'ABCEF'): 'ADD F',
 ('ABCD', 'ABCEFk'): 'ADD k',
 ('ABCD', 'ABCEFkk'): 'ADD k',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCE'): '',
 ('ABCDE', 'ABCEF'): 'ADD F',
 ('ABCDE', 'ABCEFk'): 'ADD k',
 ('ABCDE', 'ABCEFkk'): 'ADD k',
 ('ABCDEC', 'A'): 'DEL C',
 ('ABCDEC', 'AB'): 'DEL C',
 ('ABCDEC', 'AB

## Todo: Parse Solution is our homework

In [30]:
@lru_cache(maxsize=2 ** 10)
def edit_distance(string1, string2):
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)

    tail_s1 = string1[-1]       # 获取A串最后字符
    tail_s2 = string2[-1]       # 获取B串最后字符

    candidates = [
        # 给A减一位，编辑距离+1
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),
        # 给B加一位，编辑距离+1
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),
    ]

    if tail_s1 == tail_s2:
        # 如果AB最后字符相同则向前推移一位继续比较，该位不需要计算编辑距离
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        # 否则则需要进行替换操作，该位编辑距离+1
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    # 增加做过的全部操作
    candidates.append(both_forward)

    # 全部操作中次数最小的就是可实现的最小编辑距离次数及操作
    min_distance, operation = min(candidates, key=lambda x: x[0])

    solution[(string1, string2)] = operation

    return min_distance


def parse_solution(string1, string2):
    """
    解析solution结果
    :param n: 总长度
    :return:
    """
    if string1 == string2: return []
    if len(string1) == 0: return ['ADD {}'.format(i) for i in string2]
    if len(string2) == 0: return ['DEL {}'.format(i) for i in string1]

    operation = solution.get((string1, string2))

    if operation == '':
        string1, right_str = string1[:-1], string2[:-1]
        return parse_solution(string1, right_str)
    elif 'ADD' in operation:
        string2 = string2[:-1]
    elif 'DEL' in operation:
        string1 = string1[:-1]
    else:
        string1, right_str = string1[:-1], string2[:-1]
    parse_solution(string1, string2)
    return parse_solution(string1, string2) + [operation]

solution = {}
string1 = 'interpretation'
string2 = 'interkgnnblation'
min_distance = edit_distance(string1, string2)
edit_parse_solution = parse_solution(string1, string2)
print('编辑距离为：{}'.format(min_distance))
print('编辑记录为：{}'.format(edit_parse_solution))

编辑距离为：6
编辑记录为：['ADD k', 'SUB p => k', 'ADD g', 'SUB r => g', 'ADD n', 'SUB e => n', 'ADD n', 'SUB t => n', 'ADD b', 'ADD l']


# Problem Case 3: Pinyin Auto Correction Problem

In [31]:
chinese_dataset = 'article_9k.txt'

In [32]:
CHINESE_CHARATERS = open(chinese_dataset).read()

In [33]:
CHINESE_CHARATERS[:40]

'此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体'

In [34]:
import pinyin

In [35]:
pinyin.get('你好，中国',format='strip',delimiter=' ')

'ni hao ， zhong guo'

In [36]:
def chinese_to_pinyin(character):
    return pinyin.get(character, format='strip', delimiter=' ')

In [37]:
CHINESE_CHARATERS_COPYS = chinese_to_pinyin(CHINESE_CHARATERS)

In [38]:
len(CHINESE_CHARATERS_COPYS)

129433059

In [39]:
import re

In [40]:
def tokens(text):
    "List all the pinyin characters"
    return re.findall('[a-z]+',text.lower())

In [41]:
CHINESE_CHARATERS_COPYS[:100]

'ci wai zi ben zhou 6 yue 1 2 ri qi chu xiao mi shou ji 6 deng 1 5 kuan ji xing wai qi yu ji xing yi '

In [42]:
tokens(CHINESE_CHARATERS_COPYS[:-100])

['ci',
 'wai',
 'zi',
 'ben',
 'zhou',
 'yue',
 'ri',
 'qi',
 'chu',
 'xiao',
 'mi',
 'shou',
 'ji',
 'deng',
 'kuan',
 'ji',
 'xing',
 'wai',
 'qi',
 'yu',
 'ji',
 'xing',
 'yi',
 'zan',
 'ting',
 'geng',
 'xin',
 'fa',
 'bu',
 'han',
 'kai',
 'fa',
 'ban',
 'ti',
 'yan',
 'ban',
 'nei',
 'ce',
 'wen',
 'ding',
 'ban',
 'zan',
 'bu',
 'shou',
 'ying',
 'xiang',
 'yi',
 'que',
 'bao',
 'gong',
 'cheng',
 'shi',
 'ke',
 'yi',
 'ji',
 'zhong',
 'quan',
 'bu',
 'jing',
 'li',
 'jin',
 'xing',
 'xi',
 'tong',
 'you',
 'hua',
 'gong',
 'zuo',
 'you',
 'ren',
 'cai',
 'ce',
 'zhe',
 'ye',
 'shi',
 'jiang',
 'jing',
 'li',
 'zhu',
 'yao',
 'yong',
 'dao',
 'm',
 'i',
 'u',
 'i',
 'de',
 'yan',
 'fa',
 'zhi',
 'zhong',
 'm',
 'i',
 'u',
 'i',
 'qu',
 'nian',
 'yue',
 'fa',
 'bu',
 'ju',
 'jin',
 'yi',
 'you',
 'yi',
 'nian',
 'you',
 'yu',
 'ye',
 'shi',
 'shi',
 'hou',
 'geng',
 'xin',
 'huan',
 'dai',
 'le',
 'dang',
 'ran',
 'guan',
 'yu',
 'm',
 'i',
 'u',
 'i',
 'de',
 'que',
 'qie',
 'xi

In [43]:
from collections import Counter, defaultdict

In [44]:
PINYIN_COUNT = Counter(tokens(CHINESE_CHARATERS_COPYS))

In [45]:
PINYIN_COUNT

Counter({'ci': 95927,
         'wai': 131353,
         'zi': 192620,
         'ben': 55466,
         'zhou': 80326,
         'yue': 258519,
         'ri': 278379,
         'qi': 251164,
         'chu': 156005,
         'xiao': 126736,
         'mi': 40857,
         'shou': 175635,
         'ji': 645276,
         'deng': 77672,
         'kuan': 13631,
         'xing': 241741,
         'yu': 302949,
         'yi': 682476,
         'zan': 6671,
         'ting': 31732,
         'geng': 35528,
         'xin': 359619,
         'fa': 233137,
         'bu': 281533,
         'han': 39849,
         'kai': 94310,
         'ban': 83215,
         'ti': 176853,
         'yan': 125085,
         'nei': 55595,
         'ce': 30897,
         'wen': 121636,
         'ding': 60586,
         'ying': 140071,
         'xiang': 196711,
         'que': 26243,
         'bao': 132256,
         'gong': 259593,
         'cheng': 210265,
         'shi': 860634,
         'ke': 173705,
         'zhong': 409418,
     

In [46]:
all_pinyin = tokens(CHINESE_CHARATERS_COPYS)

In [47]:
all_pinyin[:20]

['ci',
 'wai',
 'zi',
 'ben',
 'zhou',
 'yue',
 'ri',
 'qi',
 'chu',
 'xiao',
 'mi',
 'shou',
 'ji',
 'deng',
 'kuan',
 'ji',
 'xing',
 'wai',
 'qi',
 'yu']

In [48]:
def correct(word):
    'Find the most possible pinyin based on edit distance'
    # Prefer edit distance 0, then 1, then 2; otherwist default to word itself
    candidates = (known(edits0(word)) or
                  known(edits1(word)) or
                  known(edits2(word)) or
                  [word])
    return max(candidates,key=PINYIN_COUNT.get)

In [49]:
def known(words):
    'Return the pinyin in our data'
    return {w for w in words if w in PINYIN_COUNT}

def edits0(word):
    'Return all strings that are zero edits away from word (i.e., just word itself).'
    return {word}

def edits2(word):
    'Return all strings that are two edits away from this pinyin.'
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [50]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

def splits(word):
    'Return a list of all possible (first, rest) pairs that comprise pinyin.'
    return [(word[:i], word[i:])
           for i in range(len(word)+1)]

def edits1(word):
    'Return all strings that are one edit away from this pinyin.'
    pairs = splits(word)
    deletes = [a+b[1:] for (a,b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a,b) in pairs if len(b) > 1]
    replaces = [a+c+b[1:] for (a,b) in pairs for c in alphabet if b]
    inserts = [a+c+b for (a,b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

In [51]:
splits('pinyin')

[('', 'pinyin'),
 ('p', 'inyin'),
 ('pi', 'nyin'),
 ('pin', 'yin'),
 ('piny', 'in'),
 ('pinyi', 'n'),
 ('pinyin', '')]

In [52]:
print(edits0('pinyin'))

{'pinyin'}


In [53]:
print(edits1('pinyin'))

{'pinzyin', 'pirnyin', 'prnyin', 'pinyin', 'pinyhin', 'pvnyin', 'pinyiy', 'pingin', 'piiyin', 'pinkyin', 'pibnyin', 'pinykn', 'pinyins', 'pinyidn', 'pinyitn', 'pmnyin', 'pninyin', 'finyin', 'piryin', 'piqyin', 'pinywin', 'piwyin', 'piunyin', 'pinydin', 'puinyin', 'jpinyin', 'xinyin', 'yinyin', 'sinyin', 'pintyin', 'phnyin', 'pinytn', 'pinyjn', 'ipinyin', 'pkinyin', 'pinykin', 'pinyinp', 'pinyan', 'pinsin', 'pinyinf', 'pinylin', 'pinyirn', 'pinqin', 'pinyil', 'pityin', 'pinpyin', 'linyin', 'pinmyin', 'winyin', 'psinyin', 'pinyinq', 'pinin', 'piuyin', 'qpinyin', 'pnyin', 'pinyhn', 'pinyiny', 'dpinyin', 'pinyign', 'pinyfn', 'penyin', 'pinuin', 'pindyin', 'pisyin', 'pinyic', 'pdinyin', 'rinyin', 'pinyzin', 'pinyni', 'panyin', 'pinyrn', 'pinyit', 'pminyin', 'pinyir', 'pinayin', 'gpinyin', 'jinyin', 'piyin', 'qinyin', 'pivnyin', 'pidyin', 'pioyin', 'piznyin', 'pinyinl', 'dinyin', 'fpinyin', 'pinyiwn', 'pdnyin', 'pinoyin', 'pinain', 'peinyin', 'pinqyin', 'piwnyin', 'pinkin', 'pjnyin', 'pinyio

# Test

In [54]:
correct('yin')

'yin'

In [55]:
correct('yign')

'ying'

In [56]:
correct('yinn')

'ying'

In [57]:
def correct_sequence_pinyin(text_pinyin):
    return ' '.join(map(correct, text_pinyin.split()))

In [58]:
correct_sequence_pinyin('zhe sih yi ge ce sho')

'zhe shi yi ge ce shi'

In [59]:
%%time
correct_sequence_pinyin('wo xiang shagn qinng hua da xue')

Wall time: 0 ns
Compiler : 236 ms


'wo xiang shang qing hua da xue'

In [60]:
correct_sequence_pinyin('wo xiang shagn qinng hau da xeu')

'wo xiang shang qing hua da xue'

# 思考题-homework？    
#### 如何在不带空格的时候完成自动修整？--> 如何完成拼音的自动分割？   
###### 提示：使用第一节课提到的语言模型!

woyaoshangqinghua
w yaoshangqinghua
wo yaoshangqinghua
woyao shangqinghua

-> DP

In [61]:
import re
import pinyin
from collections import Counter
from functools import lru_cache

In [62]:
# 读取数据并使用tokens函数提取中文
def tokens(text):   
    return ''.join(re.findall('[\u4e00-\u9fff]', text))

In [63]:
class PinyinCouter:
    """Provide the operations to counter in Chinese-Pinyin."""
    def __init__(self, dataset, counters={}):
        """
        Create the obj with two attributes.
        :param dataset: The file path
        :param counter: Defaul couter by a blank dict
        """
        self._dataset = dataset
        self._counter = counters

    @property
    def get_counter(self):
        return self._counter
    
    # 读取数据并使用tokens函数提取中文
    def read_file(self):
        with open(self._dataset) as f:  
            CHINESE_CHARATERS = f.read()
            CHINESE_CHARATERS = tokens(CHINESE_CHARATERS)
        filter_list = re.findall('[^[a-zA-Z0-9]+', CHINESE_CHARATERS.replace('\n', ""))
        filter_text = "".join(filter_list)
        return filter_text
    
    # 把读取的数据转成拼音
    @property
    def pinyin_pair(self):
        return pinyin.get(self.read_file(), format="strip", delimiter=" ")
    
    # 把拼音用Counter统计数量
    def pinyin_counter(self):
        pinyin_list = self.pinyin_pair.split(" ")
        self._counter = Counter(pinyin_list)
    
    # 生成n-gram数据，假如n为4，则把2,3,4-gram的数据整合在一起
    def expand_counter(self, n):
        """
        Expand the counter to 2-gram, 3-gram, 4-gram... till n-gram.
        :param n: the ngram model number
        :return: a final big counter
        """
        final_counter = Counter()
        for i in range(2, n+1):
            expand_list = generate_ngrams(self.pinyin_pair, i)
            final_counter += Counter(expand_list)
        return self.get_counter + final_counter

In [64]:
class EditDistance:
    """
    Provide the operation to calculate the edit distance.
    :Attribute alphabet: the Letter string between a-z.
    """
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    
    def __init__(self):
        pass

    def edits0(self, word):
        """Return all strings that are zero edits away from word (i.e., just word itself)."""
        return {word}

    def edits2(self, word):
        """Return all strings that are two edits away from this pinyin."""
        return {e2 for e1 in self.edits1(word) for e2 in self.edits1(e1)}

    def edits3(self, word):
        """Return all strings that are three edits away from this pinyin."""
        return {e3 for e2 in self.edits2(word) for e3 in self.edits1(e2)}

    def edits1(self, word):
        """Return all strings that are one edit away from this pinyin"""
        pairs = self.splits(word)
        deletes = [a + b[1:] for (a, b) in pairs if b]
        transposes = [a + b[1] + b[0] + b[2:] for (a, b) in pairs if len(b) > 1]
        replaces = [a + c + b[1:] for (a, b) in pairs for c in EditDistance.alphabet if b]
        inserts = [a + c + b for (a, b) in pairs for c in EditDistance.alphabet]
        return set(deletes + transposes + replaces + inserts)

    def splits(self, word):
        """Return a list of all possible (first, rest) pairs that comprise pinyin."""
        return [(word[:i], word[i:]) for i in range(len(word) + 1)]

In [65]:
def generate_ngrams(sentence, n):
    """
    Build the ngram model by two different method.
    Fun: Use the zip function to help us generate n-grams (faster)
    :param sentence: the initial string
    :param n: gram model number
    :return:
    """
    tokens = [token for token in sentence.split(" ") if token != ""]
    ngrams = zip(*[tokens[i:] for i in range(n)])
    
    return ["".join(ngram) for ngram in ngrams]

In [66]:
def known(words, counter):
    """
    Find the words exit in counter.
    :param words: should be a list
    :param counter: background counter
    :Return: the pinyin words we have noticed.
    """
    return {w for w in words if w in counter}

In [67]:
def correct(word, counter, ed):
    """
    Find the most possible pinyin based on edit distance.
    :param word: The incorrect sentence or word
    :param counter: The base counter data to correct the word
    :param ed: The EditDistance obj
    :return: A most likely word after correction
    """
    # Prefer edit distance 0, then 1, then 2; otherwise default to word itself.
    candidates = (known(ed.edits0(word), counter) or
                  known(ed.edits1(word), counter) or
                  known(ed.edits2(word), counter) or
                  [word]
                  )
    return max(candidates, key=counter.get)  # return the most possible

In [68]:
# To split the string after correction
def get_prob(string_list, counter, default=0):
    """
    Get the probability of a list of strings depend on Thesaurus.
    :param string_list: The origin semantics list
    :param default: The origin probability
    :param count: The Couter for search
    :return: A chance number less than 1
    """
    probility = 1
    for token in string_list:
        if token not in counter:
            probility *= default
        else:
            probility *= counter.get(token) / sum(counter.values())
    return probility

In [69]:
@lru_cache(maxsize=2 ** 10)
def best_split(string):
    """
    split a string to a few words into the dict according to the max probability.
    :param string: the phrase to be cut
    :return: the most likely percentage
    """
    # global counter
    # global split_solutions
    prob, left, right = max([(get_prob([string], counter), '', string)] + 
                            [(best_split(string[:i]) * best_split(string[i:]), 
                            string[:i], string[i:]) for i in range(1, len(string))], 
                            key=lambda x: x[0])

    split_solutions[string] = (left, right)
    return prob

In [70]:
def parse_split_solution(string):
    """
    Operate the split function.
    :param string: the phrase without splitting
    :return: a complete phrase after split
    """
    left, right = split_solutions[string]
    if not left: return [right]
    return parse_split_solution(left) + parse_split_solution(right)

In [71]:
def correct_sequence_pinyin(string, max_counter, edit_op):
    """
    Integrate the correction and split operation.
    :param string: the original phrase
    :param max_counter: the background counter
    :param edit_op: param for correct function
    :return: the final right phrase
    """
    fix_text = correct(string, max_counter, edit_op)
    prob = best_split(fix_text)
    return parse_split_solution(fix_text)

In [72]:
# 定义一些全局变量
# plit_solutions 用于best_split函数
split_solutions = {}

path = 'article_9k.txt'
count1 = PinyinCouter(path)

In [73]:
count1.pinyin_counter()
counter = count1.get_counter

In [74]:
counter.most_common(10)

[('shi', 860634),
 ('de', 809887),
 ('yi', 682476),
 ('ji', 645276),
 ('guo', 430042),
 ('zhong', 409418),
 ('zhi', 398612),
 ('xin', 359619),
 ('li', 355446),
 ('zai', 334105)]

In [75]:
%%time
# 实例化
edit_dist = EditDistance()
# 获取1,2,3,4_gram
final_counter = count1.expand_counter(5)

Wall time: 3min 14s


In [76]:
final_counter.most_common(10)

[('shi', 860634),
 ('de', 809887),
 ('yi', 682476),
 ('ji', 645276),
 ('guo', 430042),
 ('zhong', 409418),
 ('zhi', 398612),
 ('xin', 359619),
 ('li', 355446),
 ('zai', 334105)]

In [77]:
[i for i in EditDistance.edits0(edit_dist, word = 'qiguai') if i in final_counter]

['qiguai']

In [78]:
[i for i in EditDistance.edits1(edit_dist, word = 'qiguai') if i in final_counter]

['qiwuai',
 'ciguai',
 'qigudi',
 'qiguxi',
 'yiguai',
 'xiguai',
 'qigusi',
 'ziguai',
 'piguai',
 'qiguan',
 'qiguyi',
 'qiguqi',
 'qikuai',
 'qiguli',
 'jiguai',
 'qizuai',
 'qiuguai',
 'qinguai',
 'qigui',
 'qiguji',
 'qiguai',
 'qijuai',
 'miguai',
 'qigudai',
 'qigai',
 'qihuai',
 'liguai',
 'qieguai',
 'qiugai',
 'qibuai',
 'qigua',
 'qifuai',
 'quguai',
 'qiguci',
 'tiguai',
 'qiguti',
 'qiuai']

In [79]:
string1 = "woxiangshangqinghuadaxue"     # 我想上清华大学
correct_string = correct_sequence_pinyin(string1, final_counter, edit_dist)
print(correct_string)

['wo', 'xiang', 'shang', 'qing', 'hua', 'da', 'xue']


In [80]:
string1 = "xiannvhaopiaoliang"     # 仙女好漂亮
correct_string = correct_sequence_pinyin(string1, final_counter, edit_dist)
print(correct_string)

['xian', 'nv', 'hao', 'piao', 'liang']


In [81]:
string1 = "ganjeubudui"     # 感觉不对
correct_string = correct_sequence_pinyin(string1, final_counter, edit_dist)
print(correct_string)

['gan', 'jue', 'bu', 'dui']


In [82]:
string1 = "zhesihyiege"     # 这是一个
correct_string = correct_sequence_pinyin(string1, final_counter, edit_dist)
print(correct_string)

['zhe', 'shi', 'yi', 'ge']


In [83]:
string1 = "woyoaqucesuo"     # 我要去厕所
correct_string = correct_sequence_pinyin(string1, final_counter, edit_dist)
print(correct_string)

['wo', 'yo', 'a', 'qu', 'ce', 'suo']
